In [1]:
# Design the twitter app




# data structures


There are two approaches to the data structure here, a pull and a push system.

In the push system, we keep track of a user and all of their followers, and their messages, and additionally a user and their feed.

## push archetecture

In this method, when a new message is published by a user, we need to look up their followers and push the message to the followers feeds.

User followers

    User, follower, metadata
    
There is a inque row for each (user, follower) pair, meaning a user has many rows, one for each follower.

User feed

    User, message_id

There is a unique row for each (user, message_id) and we construct the feed by selecting all the rows with that user.

Messages

    message_id, message, metadata
    
The problem here is that if a user is very popular, i.e. has millions of followers, publishing to followers feed list will involve millions of writes.  The good part of this approach is that constructing feeds is realatively cheep.  We simply get their messages

Notice I haven't mentioned timestamps, or message sorting.  We can include the message timestamp in the message table row, and order them, or we can build time into the order of the message_ids to save some time in sorting.  The second approach works straight away if we are constructing a purely time-ordered feed.  If we have some kind of formula or model for relevance, and order by relevance, we need to have access to timestamps, which we can either keep in the message table as a column, or we can build a system for extracting the timestamp from the message id.


## pull archetecture

In the pull system, involves keeping track of all the users a user follows.

User following

    User, User (or user list)
    
Where for every user we keep track of who they're following.

User messages

    User, message_id

Messages

    message_id, message, metadata

Now, to construct feeds, we need to get all the other users a user is following, then grab their messages and put them in order.
Encoding message timestamp in the message_id helps here too.

### Performance improvements

To speed up the gathering of messages, we can keep a message cache (LRU or something similar).  This will help us with very popular messages and very recent messages.

To improve experience at log-in, we can keep a table of every user's most recent feed, and get it immediately on log-in.  We'll set a size limit of this feed to keep.  This will get users old messages already in their feed, and allow us some time to contruct a feed of new messages (or old feed messages if they're looking backwards).

Deleted messages are handles by dropping them when we try to look them up.  For the cached feeds, we can validate the messages (see if they still exist, but not read them entirely.  If they don't exist anymore, we can drop them from the cached feed before returning them.

